In [18]:
# multivariate multi-step stacked lstm example
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Dropout
import optuna

In [2]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out-1
		# check if we are beyond the dataset
		if out_end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [3]:
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

In [4]:
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

In [7]:
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
dataset

array([[ 10,  15,  25],
       [ 20,  25,  45],
       [ 30,  35,  65],
       [ 40,  45,  85],
       [ 50,  55, 105],
       [ 60,  65, 125],
       [ 70,  75, 145],
       [ 80,  85, 165],
       [ 90,  95, 185]])

In [8]:
# choose a number of time steps
n_steps_in, n_steps_out = 3, 2

In [9]:
# covert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)
X

array([[[10, 15],
        [20, 25],
        [30, 35]],

       [[20, 25],
        [30, 35],
        [40, 45]],

       [[30, 35],
        [40, 45],
        [50, 55]],

       [[40, 45],
        [50, 55],
        [60, 65]],

       [[50, 55],
        [60, 65],
        [70, 75]],

       [[60, 65],
        [70, 75],
        [80, 85]]])

In [10]:
y

array([[ 65,  85],
       [ 85, 105],
       [105, 125],
       [125, 145],
       [145, 165],
       [165, 185]])

In [11]:
X.shape
# 6 items de 3 pasos de entrada con 2 variables

(6, 3, 2)

In [12]:
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]

In [13]:
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps_in, n_features)))
model.add(LSTM(100, activation='relu'))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')

2023-11-25 12:13:09.629677: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2023-11-25 12:13:09.629695: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: Legion-Gero
2023-11-25 12:13:09.629698: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: Legion-Gero
2023-11-25 12:13:09.629779: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 525.147.5
2023-11-25 12:13:09.629787: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 525.147.5
2023-11-25 12:13:09.629788: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 525.147.5


In [ ]:
# fit model
model.fit(X, y, epochs=200, verbose=1)

In [16]:
# demonstrate prediction
x_input = array([[70, 75], [80, 85], [90, 95]])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[185.76114 206.64917]]


In [19]:
yhat[0][0]

185.76114

# BO

In [50]:
def train_model(n_layers, n_neurons, dropout_threshold, n_steps, X, y, embudo, embudo_2, embudo_3):
    # Train a single model where product_id is part of the input
    print(f"Training model with {n_layers} layers, {n_neurons} neurons, {n_steps} steps, {dropout_threshold} dropout, {embudo} embudo, {embudo_2} embudo_2 y {embudo_3} embudo_3.")

    # Number of features 
    n_features = X.shape[2]

    # Define the LSTM model
    if n_layers == 1:
        model = Sequential()
        model.add(LSTM(n_neurons, activation="relu", input_shape=(n_steps, n_features)))
        model.add(Dense(n_steps_out))
        model.compile(optimizer="adam", loss="mse")
    elif n_layers == 2:
        model = Sequential()
        model.add(LSTM(n_neurons, activation="relu", return_sequences=True, input_shape=(n_steps, n_features)))
        model.add(Dropout(dropout_threshold))
        model.add(LSTM(int(n_neurons // embudo), activation="relu"))
        model.add(Dense(n_steps_out))
        model.compile(optimizer="adam", loss="mse")
    elif n_layers == 3:
        model = Sequential()
        model.add(LSTM(n_neurons, activation="relu", return_sequences=True, input_shape=(n_steps, n_features)))
        model.add(Dropout(dropout_threshold))
        model.add(LSTM(int(n_neurons // embudo), activation="relu", return_sequences=True))
        model.add(Dropout(dropout_threshold))
        model.add(LSTM(int(n_neurons // embudo_2), activation="relu"))
        model.add(Dense(n_steps_out))
        model.compile(optimizer="adam", loss="mse")
    elif n_layers == 4:
        model = Sequential()
        model.add(LSTM(n_neurons, activation="relu", return_sequences=True, input_shape=(n_steps, n_features)))
        model.add(Dropout(dropout_threshold))
        model.add(LSTM(int(n_neurons // embudo), activation="relu", return_sequences=True))
        model.add(Dropout(dropout_threshold))
        model.add(LSTM(int(n_neurons // embudo_2), activation="relu", return_sequences=True))
        model.add(Dropout(dropout_threshold))
        model.add(LSTM(int(n_neurons // embudo_3), activation="relu"))
        model.add(Dense(n_steps_out))
        model.compile(optimizer="adam", loss="mse")
    else:
        raise ValueError("n_layers must be 1, 2, 3 or 4")


    # Fit the model
    model.fit(X, y, epochs=200, verbose=0)
    return model

In [47]:
def make_predictions(n_steps, model, X_input):
    n_features = X_input.shape[2]

    # Make prediction
    yhat = model.predict(X_input, verbose=1)

    return yhat[0][0], yhat[0][1]

In [22]:
def calculate_score(prediction_1, prediction_2, actual_1, actual_2):
    predicted = [prediction_1, prediction_2]
    real_values = [actual_1, actual_2]
    error = 0
    real_sum = sum(real_values)
    for i in range(len(predicted)):
        error += abs(predicted[i] - real_values[i])
    return error / real_sum

In [52]:
def objective(trial):
    # Define the search space using Optuna
    n_layers = trial.suggest_int('n_layers', 1, 4)
    n_neurons = trial.suggest_int('n_neurons', 20, 1000)
    n_steps_in = trial.suggest_int('n_steps', 1, 5)
    embudo = trial.suggest_float('embudo', 1, 2.5)
    embudo_2 = trial.suggest_float('embudo_2', 1, 3.5)
    embudo_3 = trial.suggest_float('embudo_3', 1, 4.5)
    dropout_threshold = trial.suggest_float('dropout_threshold', 0.1, 0.66)

    # Your existing logic to train and evaluate the model
    X, y = split_sequences(dataset, n_steps_in, n_steps_out)

    x_input = dataset[-n_steps_in:, :-1]
    
    # demonstrate prediction
    x_input = x_input.reshape((1, n_steps_in, n_features))

    # X_test, y_test = split_sequences(test_df.values, n_steps)
    # X_predict, y_predict = split_sequences(predict_df.values, n_steps)

    model = train_model(n_layers, n_neurons, dropout_threshold, n_steps_in, X, y, embudo, embudo_2, embudo_3)
    prediction_1, prediction_2 = make_predictions(n_steps_in, model, x_input)
    score = calculate_score(prediction_1, prediction_2, 185, 205)
    print(f"Score: {score}")
    return score

In [53]:
# Create a study object and specify the optimization direction
study = optuna.create_study(direction='minimize', study_name="lstm_8_BO_test")

# Perform optimization
study.optimize(objective, n_trials=50)

# Extract the best trial
best_trial = study.best_trial

print('Best trial:')
for key, value in best_trial.params.items():
    print(f"{key}: {value}")

[I 2023-11-25 12:59:19,286] A new study created in memory with name: lstm_8_BO_test


{'out': None}
(1, 4)
{'out': None}
(20, 1000)
{'out': None}
(1, 5)
Training model with 3 layers, 402 neurons, 3 steps, 0.35338190784420687 dropout, 2.017619217713375 embudo, 2.1002354324523758 embudo_2 y 3.649998412630036 embudo_3.
1/1 [==============================] - 0s 199ms/step


[I 2023-11-25 12:59:23,080] Trial 0 finished with value: 0.046408472305689105 and parameters: {'n_layers': 3, 'n_neurons': 402, 'n_steps': 3, 'embudo': 2.017619217713375, 'embudo_2': 2.1002354324523758, 'embudo_3': 3.649998412630036, 'dropout_threshold': 0.35338190784420687}. Best is trial 0 with value: 0.046408472305689105.


Score: 0.046408472305689105
{'out': None}
(1, 4)
{'out': None}
(20, 1000)
{'out': None}
(1, 5)
Training model with 3 layers, 381 neurons, 5 steps, 0.3629775449049243 dropout, 1.030186743093878 embudo, 3.399441482134514 embudo_2 y 1.0598013854062998 embudo_3.
1/1 [==============================] - 0s 177ms/step


[I 2023-11-25 12:59:26,442] Trial 1 finished with value: 0.16594050480769232 and parameters: {'n_layers': 3, 'n_neurons': 381, 'n_steps': 5, 'embudo': 1.030186743093878, 'embudo_2': 3.399441482134514, 'embudo_3': 1.0598013854062998, 'dropout_threshold': 0.3629775449049243}. Best is trial 0 with value: 0.046408472305689105.


...

Score: 0.09702273637820513
{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'out': None}
(0.5714285714285714, 4.0)
{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'out': None}
(0.08888888888888889, 4.0)
{'decimals': 0, 'out': None}
()
{'out': None}
(1, 4)
{'decimals': 0, 'out': None}
()
{'out': None}
(1, 4)
{'axis': None, 'out': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'out': None}
(140.14285714285714, 981.0)
{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'out': None}
(21.8, 981.0)
{'decimals': 0, 'out': None}
()
{'out': None}
(20, 1000)
{'decimals': 0, 'out': None}
()
{'out': None}
(20, 1000)
{'axis': None, 'out': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'out': None}
(0.7142857142857143, 5.0)
{'axis': -1, 'kind': None, 'order': None}
()
{'axi

[I 2023-11-25 13:01:50,033] Trial 48 finished with value: 0.0031844701522435897 and parameters: {'n_layers': 1, 'n_neurons': 959, 'n_steps': 2, 'embudo': 1.9464575726009246, 'embudo_2': 1.3585688731290242, 'embudo_3': 3.8566050019895393, 'dropout_threshold': 0.5872723752919715}. Best is trial 28 with value: 0.0006374261318108974.


Score: 0.0031844701522435897
{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'out': None}
(0.5714285714285714, 4.0)
{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'out': None}
(0.08695652173913043, 4.0)
{'decimals': 0, 'out': None}
()
{'out': None}
(1, 4)
{'decimals': 0, 'out': None}
()
{'out': None}
(1, 4)
{'axis': None, 'out': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'out': None}
(140.14285714285714, 981.0)
{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'out': None}
(21.32608695652174, 981.0)
{'decimals': 0, 'out': None}
()
{'out': None}
(20, 1000)
{'decimals': 0, 'out': None}
()
{'out': None}
(20, 1000)
{'axis': None, 'out': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'out': None}
(0.7142857142857143, 5.0)
{'axis': -1, 'kind': None, 'order':

[I 2023-11-25 13:01:51,064] Trial 49 finished with value: 0.005620946639623398 and parameters: {'n_layers': 1, 'n_neurons': 318, 'n_steps': 2, 'embudo': 1.612489569111844, 'embudo_2': 1.3310322426904815, 'embudo_3': 3.8657108280718675, 'dropout_threshold': 0.5881591768786434}. Best is trial 28 with value: 0.0006374261318108974.


Score: 0.005620946639623398
Best trial:
n_layers: 1
n_neurons: 927
n_steps: 2
embudo: 2.1044756432966993
embudo_2: 1.2174513395275657
embudo_3: 2.8773715429726323
dropout_threshold: 0.545209565034454
